# Zadania:

1. przeczytaj materiały dotyczące pulp ze strony internetowej https://coin-or.github.io/pulp/main/index.html oraz przejdź przez blending problem (przepisz do swojego notebooka)

2. w blending problem dodaj zmienną decyzyjną. Stwórz zmienną która pojawi się w końcowym składzie produktu (tj. wartość zmiennej będzie miała niezerową postać)

3. Dodaj ograniczenie co do zawartości witaminy B 

Zawartość:

| Nazwa      | zawartość ug/100g |
|------------|-------------------|
| chicken    | 0.3               |
| beef       | 2.6               |
| mutton     | 2.6               |
| rice       | 0                 |
| wheat bran | 0                 |
| gel        | 0                 |
| sardines   | 8.9               |

Kot ma zjeść minimum 0.24 ug na kilogram masy więc około
0.8 ug dziennie

4. Na końcu ma pojawiać się zdanie podsumowujące wynik, w stylu:
Końcowy produkt będzie kosztować {}, będzie miał w składzie {nazwa}: {} %, {nastepna nazwa}: {} %

Takie zdanie można skleić z części generowanych automatycznie:


In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp

In [1]:
import pulp
print(pulp.__version__)

2.3.1


## Simplified Formulation
Zadanie polegało na rozwiazaniu problemu wytwarzania karmy najtaniej jak jest to możliwe, jednoczesnie spełniając warunki jakie narzucają ograniczenia co do wartości odżywczych. <br />
**Zmienne decyzyjne:**

x1 = kurczak <br />
x2 = wołowina <br />
x3 = sardynki <br />

**Funkcja celu:**

Minimalizacja kosztów wytowrzenia karmy

min(0.013x1 + 0.008x2 + 0.003x3)

**Ograniczenia:**

*   Procentówa suma składników == 100%
*   Proteiny >= 8g/100g
*   Tłuszcz >= 6g/100g
*   Błonnik <= 2g/100g
*   Sól <= 0.4g/100g
*   Witamina B >= 0.8ug/100g

In [2]:
#Simplified Formulation

# Import PuLP modeler functions
from pulp import *

# Create the 'prob' variable to contain the problem data
prob = LpProblem("The Whiskas Problem",LpMinimize)

# The 2 variables Beef and Chicken are created with a lower limit of zero
x1=LpVariable("Chicken",0,None,LpInteger)
x2=LpVariable("Beef",0)
x3=LpVariable("Sardines",0)

# The objective function is added to 'prob' first
prob += 0.013*x1 + 0.008*x2 + 0.003*x3, "Total Cost of Ingredients per can"

# The five constraints are entered
prob += x1 + x2 + x3 == 100, "PercentagesSum"
prob += 0.100*x1 + 0.200*x2 + 0.050*x3 >= 8.0, "ProteinRequirement"
prob += 0.080*x1 + 0.100*x2 + 0.020*x3 >= 6.0, "FatRequirement"
prob += 0.001*x1 + 0.005*x2 + 0.100*x3 <= 2.0, "FibreRequirement"
prob += 0.002*x1 + 0.005*x2 + 0.007*x3 <= 0.4, "SaltRequirement"
prob += 0.003*x1 + 0.002*x2 + 0.400*x3 >= 0.8, "VitaminRequirement"

# The problem data is written to an .lp file
prob.writeLP("WhiskasModel.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print ("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print (v.name, "=", v.varValue)


print ("Końcowy produkt będzie kosztować {}, bedzie miał w składzie {}={}%, {}={}%, {}={}%. \n".format(value(prob.objective), x1.name, x1.value(), x2.name, x2.value(), x3.name, x3.value()))

# The optimised objective function value is printed to the screen
#print ("Total Cost of Ingredients per can = ", value(prob.objective))

Status: Optimal
Beef = 37.5
Chicken = 45.0
Sardines = 17.5
Końcowy produkt będzie kosztować 0.9375, bedzie miał w składzie Chicken=45.0%, Beef=37.5%, Sardines=17.5%. 



/home/arek/miniconda3/lib/python3.8/site-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## Full Formulation
Analogiczne zadanie do poprzedniego, zrealizowane w bardizej rozbudowanej formie, z dokładniej okreslonymi danymi i stworzonymi ogrnaiczeniami. <br />


In [6]:
#Full Formulation

# Import PuLP modeler functions
from pulp import *

Ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL', 'SARDINES']

# A dictionary of the costs of each of the Ingredients is created
costs = {'CHICKEN': 0.013, 
         'BEEF': 0.008, 
         'MUTTON': 0.010, 
         'RICE': 0.002, 
         'WHEAT': 0.005, 
         'GEL': 0.001,
         'SARDINES': 0.300}


# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100, 
                  'BEEF': 0.200, 
                  'MUTTON': 0.150, 
                  'RICE': 0.000, 
                  'WHEAT': 0.040, 
                  'GEL': 0.000,
                  'SARDINES': 0.050}

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000,
              'SARDINES': 0.020}

# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000,
                'SARDINES': 0.100}

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000,
               'SARDINES': 0.007}

# A dictionary of the Vitamin percent in each of the Ingredients is created
vitaminPercent = {'CHICKEN': 0.004, 
               'BEEF': 0.004, 
               'MUTTON': 0.004, 
               'RICE': 0.002, 
               'WHEAT': 0.004, 
               'GEL': 0.000,
               'SARDINES': 0.400}

# Create the 'prob' variable to contain the problem data
prob = LpProblem("The Whiskas Problem", LpMinimize)
# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
ingredient_vars = LpVariable.dicts("Ingr",Ingredients,0)
# The objective function is added to 'prob' first
prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]), "Total Cost of Ingredients per can"
# The five constraints are added to 'prob'
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"
prob += lpSum([vitaminPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 0.8, "VitaminRequirement"

prob.writeLP("WhiskasModel.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print ("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print (v.name, "=", v.varValue)
    

sklejone_produkt_cena = [":".join((name, str(v.varValue))) for name,v in zip(['BEEF', 'CHICKEN','GEL', 'MUTTON', 'RICE','SARDINES', 'WHEAT'],prob.variables())]
sklejona_calosc = "%, ".join(sklejone_produkt_cena)
print()
print ("Końcowy produkt będzie kosztować",value(prob.objective),"\nBedzie miał w składzie",sklejona_calosc,"\n")

# The optimised objective function value is printed to the screen
#print ("Total Cost of Ingredients per can = ", value(prob.objective))

Status: Optimal
Ingr_BEEF = 58.156817
Ingr_CHICKEN = 0.0
Ingr_GEL = 24.751024
Ingr_MUTTON = 0.0
Ingr_RICE = 15.75249
Ingr_SARDINES = 1.3396694
Ingr_WHEAT = 0.0

Końcowy produkt będzie kosztować 0.92341136 
Bedzie miał w składzie BEEF:58.156817%, CHICKEN:0.0%, GEL:24.751024%, MUTTON:0.0%, RICE:15.75249%, SARDINES:1.3396694%, WHEAT:0.0 

